In [1]:
import sys
import os
sys.path.append("../../src/")
sys.path.append("../../utilities/")
import dataset_class
from dataset_class import dataset
import data_utils
import evaluation_utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from hmmlearn import hmm
import mrf_util
from mrf import MarkovRandomField
from GMM_GBR import GMM_MP

/home/nedooshki/miniconda3/envs/IChDA/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
gene_expression_path = '../../data/GM12878/hg19/gene_expression/GM12878_genes_RPKM.txt'
RT_path = '../../data/GM12878/hg19/RT/six_phase/six_phases.txt'
CTCF_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/CTCF/lifted_data'
RNAPII_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/RNAPII/lifted_data'

In [2]:
signals_dir = '../../data/GM12878/hg19/genomic-assays/bin_100000/'
CTCF_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/CTCF/lifted_data'
RNAPII_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/RNAPII/lifted_data'
gene_expression_path = '../../data/GM12878/hg19/gene_expression/GM12878_genes_RPKM.txt'
RT_path = '../../data/GM12878/hg19/RT/six_phase/six_phases.txt'
annotation_dir = "../../data/GM12878/hg19/res100000_datasets/annotations"
functional_signals = np.loadtxt('../../data/GM12878/hg19/res100000_datasets/signals.txt')
structural_signals = np.loadtxt('../../data/GM12878/hg19/res100000_datasets/structural_signals.txt')
annot_dir = os.path.join(annotation_dir, 'k8')

In [3]:
GM12878_signals_names = ['DNase', 'H3K4me2', 'H3K27ac', 'H2A.Z', 'H3K9me3', 'H3K4me1', 'H3K27me3', 'H3K9ac', 'H3K4me3', 'H3K36me3', 'H3K79me2', 'H4K20me1']
datasets = {}
res = 100000
GM12878_dataset = dataset('GM12878', 'hg19', res, '../../data/GM12878/hg19/Hi-C/GSE63525_GM12878_insitu_primary_30.hic',
                 '../../utilities/juicer_tools_1.22.01.jar', '../../data/GM12878/hg19/Hi-C/res{}'.format(res),
                 GM12878_signals_names, '../../data/GM12878/hg19/genomic-assays/bin_{}/'.format(res), 
                  '../../data/GM12878/hg19/res{}_datasets/'.format(res),
                  '../../data/', '../../utilities/config.json')
GM12878_dataset.load_pos2ind_and_ind2pos_maps('../../data/valid_bins/hg19/GM12878_{}_bins.txt'.format(res))

In [ ]:
k = 8
signals_path = "../../data/GM12878/hg19/res100000_datasets/signals.txt"
bins_path = "../../data/GM12878/hg19/res100000_datasets/bin.txt"
chain_path = '../../data/GM12878/hg19/res100000_datasets/interactions/chain_interactions.txt'
oe_intra_hic_path = '../../data/GM12878/hg19/res100000_datasets/interactions/oe_significant_intra_interactions_b2000.txt'
annotation_dir = "../../data/GM12878/hg19/res100000_datasets/annotations/"
annot_dir = os.path.join(annotation_dir, 'k{}'.format(k))

# alternatives

In [ ]:
'''
gmm_mp_args = {"i": signals_path, "e": chain_path, "g": bins_path, "r": 1, "n": k, 
              "il": 5, "ml": 10, "w": 10, "o": 'GM12878_res100000_mp', "mp": '../../alternatives/segway-gbr-master/measureProp/MP_large_scale'}
gmm_mp = GMM_MP(gmm_mp_args)
gmm_mp.write_mp_graph()
gmm_mp.GMM_train()
gmm_mp.GMM_inference(0)
for l in range(10):
    gmm_mp.write_mp_post(l+1)
    gmm_mp.run_measure_prop(l+1)
    gmm_mp.read_mp_post(l+1)
    gmm_mp.GMM_inference(l+1)
GM12878_dataset.write_annotation(gmm_mp.iters_labels[-1], 'GMM_GBR_chain', annot_dir)


gmm_mp_args = {"i": signals_path, "e": oe_intra_hic_path, "g": bins_path, "r": 1, "n": k, 
              "il": 5, "ml": 10, "w": 10, "o": 'GM12878_res100000_mp', "mp": '../../alternatives/segway-gbr-master/measureProp/MP_large_scale'}
gmm_mp = GMM_MP(gmm_mp_args)
gmm_mp.write_mp_graph()
gmm_mp.GMM_train()
gmm_mp.GMM_inference(0)
for l in range(10):
    gmm_mp.write_mp_post(l+1)
    gmm_mp.run_measure_prop(l+1)
    gmm_mp.read_mp_post(l+1)
    gmm_mp.GMM_inference(l+1)
GM12878_dataset.write_annotation(gmm_mp.iters_labels[-1], 'GMM_GBR_intra', annot_dir)

'''
mrf_signals = mrf_util.readData(signals_path)
mrf_bins = mrf_util.readBedGraph(bins_path)
n = mrf_signals.shape[0]

# SPIN with chain structure
mrf_hic = mrf_util.create_hic_matrix([chain_path], n, True)
hmrf = MarkovRandomField(n, mrf_hic, mrf_signals, 'mrf', k, 5, 0.01)
hmrf.init_gmm()
hmrf.init_trans()
hmrf.solve()
GM12878_dataset.write_annotation(hmrf.iters_labels[-1], 'SPIN_chain', annot_dir)


# SPIN with chain + intra-chromosomal O/E structure
mrf_hic = mrf_util.create_hic_matrix([chain_path,oe_intra_hic_path], n, True)
hmrf = MarkovRandomField(n, mrf_hic, mrf_signals, 'mrf', k, 5, 0.01)
hmrf.init_gmm()
hmrf.init_trans()
hmrf.solve()
GM12878_dataset.write_annotation(hmrf.iters_labels[-1], 'SPIN_chain_intra', annot_dir)

In [ ]:
stats_df = []
for method in ['HMM_functional', 'HMM_structural', 'HMM_combined', 'GMM_GBR_chain', 'GMM_GBR_intra', 'SPIN_chain', 'SPIN_intra']:
   
    annot_path = os.path.join(annot_dir, '{}_annotation.txt'.format(method))
    stats = evaluation_utils.stats(annot_path, 100000, gene_expression_path, RT_path, 
                                   CTCF_bedpe_path, RNAPII_bedpe_path)
    stats['method'] = method
    stats_df.append(stats)

stats_df = pd.DataFrame(stats_df)